# Q1

In [292]:
import os
import tarfile
from six.moves import urllib
import pandas as pd
import numpy as np

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
        
    tgz_path = os.path.join(housing_path, "housing.tgz")
    if not os.path.isfile(tgz_path): #download data if not already there
        urllib.request.urlretrieve(housing_url, tgz_path)
        
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [293]:
from sklearn.model_selection import StratifiedShuffleSplit
# deal with skewness
# Divide by 1.5 to limit the number of income categories
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Label those above 5 as 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

    - Estimators: any object estimating some parameters. In the above, the imputer is an estimator. Estimators need to have a fit() method which take the dataset as input. Any other parameters are considered as hyperparameters, e.g. the strategy hyperparameter in the imputer

    - Transformers: these are estimators which can transofrm the dataset. They need to implement the transform() method. All transformers also has a fit_transform() method equivalent to calling fit() and then transform(). Sometimes the fit_transform() method is better optimized for efficiency so usually best to call it instead of fit() and then transform(). The imputer above is actually a transformer.

    - Predictors: these are estimators which can make predictions. LinearRegression model is a predictor. Predictors must implement a predict() method. They also have a score() method that measures the quality of the predictions.


In [294]:
# data preprocessing
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()
# there are na values
incomplete_rows = housing[housing.isnull().any(axis=1)] # Take out those with na value
incomplete_rows.shape

(158, 9)

## Deal with NA values

In [295]:
from sklearn.impute import SimpleImputer
# replace na by median
housing_num = housing.drop("ocean_proximity", axis=1) # median only works with numerical attributes
# the followings are commented since later we use pipeline to solve na problems all together
# imputer = SimpleImputer(strategy="median")
# imputer.fit(housing_num)
# print(imputer.statistics_)
# print(housing_num.median().values)
# X = imputer.transform(housing_num)
# housing_tr = pd.DataFrame(X, columns=housing_num.columns, index = list(housing.index.values))
# # show that na values has been replaced
# housing_tr.loc[sample_incomplete_rows.index.values]
# # back to index starting form 1
# housing_tr = pd.DataFrame(X, columns=housing_num.columns)
# housing_tr.head()

## Deal with the categorical variables

In [296]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
housing_cat = housing["ocean_proximity"]
housing_cat_encoded = encoder.fit_transform(housing_cat)
# print(housing_cat_encoded)
# print(encoder.classes_) ['<1H OCEAN' 'INLAND' 'ISLAND' 'NEAR BAY' 'NEAR OCEAN']
# However the LabelEncoder above assumes order. E.g. label 0 is < than label 4. This should not be the case for
# the encoder classes for the housing dataset. That is why we use instead a one-hot-encoder

# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder(categories='auto')
# housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1))
# housing_cat_1hot # note that the output is a sparse matrix
# housing_cat_1hot.toarray() can convert back to numpy array... but this takes lots of memory!

## Use pipeline 

In [297]:
from sklearn.base import BaseEstimator, TransformerMixin
# column index
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
# add some features 
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

# attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
# housing_extra_attribs = attr_adder.transform(housing.values)
# housing_extra_attribs = pd.DataFrame(housing_extra_attribs, 
#                                      columns=list(housing.columns) + ["rooms_per_household", "population_per_household"])
# housing_extra_attribs

In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

# Create a class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values
    
# LabelBinarizer is same as OneHotEncoder but fit_transform() equivalent to fit_transform().to_array() of OneHotEncoder
# unfortunately LabelBinarizer isn't pipeline friendly so we'll have to extend it as below:
from sklearn.preprocessing import LabelBinarizer 
class PipelineFriendlyLabelBinarizer(LabelBinarizer):
    def fit_transform(self, X, y=None):
        return super(PipelineFriendlyLabelBinarizer, self).fit_transform(X)
    
ppflb = PipelineFriendlyLabelBinarizer()
housing_cat_1hot_lb = ppflb.fit_transform(housing_cat_encoded.reshape(-1,1))
housing_cat_1hot_lb[:5]

NameError: name 'housing_cat_encoded' is not defined

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# let's now combine the numerical and categorical pipelines
num_attribs = list(housing_num) # columns 
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('label_binarizer', PipelineFriendlyLabelBinarizer()),
    ])

# and concatenate them with FeatureUnion class
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

# this is the final transformation result!
housing_prepared = full_pipeline.fit_transform(housing)

In [2]:
# (a)
import statsmodels.api as sm
import matplotlib.pyplot as plt

class LinRegStatsmodels():
    def __init__(self):
        self.models = []
    def fit(self,X,y):
        X = sm.add_constant(X)
        model = sm.OLS(y,X)
        res = model.fit()
        self.models.append(res)
    def predict(self,data): 
        return self.models[-1].predict(sm.add_constant(data,has_constant='add'))
    
# (b)
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinRegStatsmodels())
    ])
# A full pipeline with both preparation and prediction
full_pipeline_with_predictor.fit(housing, housing_labels)
#####################################################################
# pay attention to the difference between fit_transform and transform
# no need to fit on the test data
some_data = housing.iloc[:5]
processed_data = full_pipeline.transform(some_data)
pred = full_pipeline_with_predictor[1].models[-1].predict(sm.add_constant(processed_data,has_constant='add'))
print(full_pipeline_with_predictor.predict(some_data) == pred)

residual = full_pipeline_with_predictor[1].models[-1].resid
plt.scatter(range(len(residual)),residual,s=0.1)
plt.ylabel("Residual")
plt.show()

print(full_pipeline_with_predictor[1].models[-1].summary())
print('###########################################################################################')
print(full_pipeline_with_predictor[1].models[-1].get_robustcov_results().summary())

NameError: name 'Pipeline' is not defined

(c)
Under OLS, we can see that with 95% confidence level, x4(total_rooms), x10(rooms_per_household),x13(dummy variable for the categorical variable ocean_proximity) are not statistically significant. \
Under OLS with robust standard error, with 95% confidence level,x4(total_rooms), x5(total_bedrooms), x10(rooms_per_household) are not statistically significant. \
Although the residual plot does not show any obvious pattern of heteroskedasticity, the test results actually changes under robust standard errors. That being said, there is heteroskedasticity in the dataset.

In [301]:
# (d)
new_data = pd.DataFrame({"longitude":[-118.8],"latitude":[34.19],"housing_median_age":[4],"total_rooms":[15572],"total_bedrooms":[2222],"population":[5495],
                      "households":[2152],"median_income":[housing["median_income"].median()],"ocean_proximity":["<1H OCEAN"]})
print(full_pipeline_with_predictor.predict(new_data))

[241862.50684923]


Problems with the given data: \
(1) The data given does not have column named "median_income", so we suppose that the value is the median of the value of "median_income" in the original dataset. \
(2) The "ocean_proximity" give is "1H OCEAN", which not belongs to any class of the categorical variable "ocean_proximity". We only has "<1H OCEAN", so we assume that it is a typo and should be "<1H OCEAN" instead.

In [302]:
# (d)
# 95% Confidence Interval for the predicted value:
# OLSResults.get_prediction() method will return a linear_model.PredictionResults object
# The prediction results instance contains prediction and prediction variance 
# and can on demand calculate confidence intervals and summary tables for the prediction of the mean and of new observations.
# OLS:
processed_new_data = full_pipeline.transform(new_data)
pred_model = full_pipeline_with_predictor[1].models[-1].get_prediction(sm.add_constant(processed_new_data, has_constant='add'))
confidence_interval = list(pred_model.conf_int(0.05)[0])
print("OLS C.I. ", confidence_interval)
robust_pred_model =  full_pipeline_with_predictor[1].models[-1].get_robustcov_results().get_prediction(sm.add_constant(processed_new_data, has_constant='add'))
robust_confidence_interval = list(robust_pred_model.conf_int(0.05)[0])
print("OLS + robust standard errors C.I. ", robust_confidence_interval)

OLS C.I.  [106882.4895938003, 376842.52410466364]
OLS + robust standard errors C.I.  [106179.44452046955, 377545.5691779944]


(d)\
For the point estimator, OLS and OLS + robust standard errors produce the same estimated value. \
For the confidence interval, I would choose OLS + robust standard errors since it can relieve heteroskedasticity. In terms of model criteria, they have same $R^2$, AIC, BIC, etc. The only differences lie in statistical inference. The F statistic of robust OLS is much more larger than that of OLS.

# Q2


In [304]:
ff5_path = './datasets/F-F_Research_Data_5_Factors_2x3.csv'
equityPrice_path = './datasets/QMNIX.csv'
ff5 = pd.read_csv(ff5_path, dtype={'Month':str})
ff5 = ff5.dropna()
equityPrice = pd.read_csv(equityPrice_path,usecols=['Date', 'Adj Close']).rename(columns={'Adj Close':'AdjClose'})
equityPrice.Date = equityPrice.Date.apply(lambda x:''.join(x.split('-')))
equityPrice['Date'] = equityPrice.Date.apply(lambda x:x[:-2])
equityPrice = equityPrice.dropna()
months = equityPrice["Date"].unique().tolist()
Ret = {}
for month in months:
    if month in [months[0], months[-1]]:
        continue
    this_month = equityPrice[equityPrice["Date"] == month]
    preMonth = [i for i in months if int(i) < int(month)][-1]
    pre_month = equityPrice[equityPrice["Date"] == preMonth]
    first = pre_month["AdjClose"].iloc[-1]
    last = this_month["AdjClose"].iloc[-1]
    Ret[month] = np.log(last/first)
ff5['RET'] = ff5.Month.apply(lambda x:Ret[x] if x in Ret else np.nan) - ff5.RF/100
ff5 = ff5[['Month','Mkt-RF','SMB','HML','RMW','CMA','RET']]
ff5 = ff5.dropna().reset_index(drop=True)

In [311]:
# Regression
# OLS
import statsmodels.api as sm
X = sm.add_constant(ff5[['Mkt-RF','SMB','HML','RMW','CMA']])
model = sm.OLS(ff5['RET'], X)
result = model.fit()
print(result.summary())
print("#####################################################################################")
# OLS + Robust standard errors
print(result.get_robustcov_results().summary())

                            OLS Regression Results                            
Dep. Variable:                    RET   R-squared:                       0.396
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     12.98
Date:                Sat, 30 Sep 2023   Prob (F-statistic):           1.02e-09
Time:                        21:31:43   Log-Likelihood:                 252.11
No. Observations:                 105   AIC:                            -492.2
Df Residuals:                      99   BIC:                            -476.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0028      0.002      1.217      0.2

Again, we can find that two models have same values for criteria that have nothing to do with inference. \
The F and t statistics have not changed much under heteroskedastic standard errors, and still, only HML is statistically significant with 95% confidence level.